# Chapter 3: Introduction to Symforce Symbolic Computation

## 🎯 학습 목표

이 챕터에서는 Symforce의 핵심인 symbolic computation을 배우고, 자동 미분의 강력함을 체험합니다.

- Symforce의 symbolic 변수와 표현식
- 자동 미분(Automatic Differentiation)의 원리
- SE(3) 연산의 symbolic 표현
- Jacobian 자동 계산
- Codegen을 통한 최적화

## 📚 이론적 배경

### Symbolic Computation이란?

Symbolic computation은 수식을 기호적으로 다루는 방법입니다:
- 수치 계산: `f(2) = 2² + 3×2 = 10`
- 기호 계산: `f(x) = x² + 3x` → `f'(x) = 2x + 3`

### Symforce의 장점

1. **자동 미분**: 복잡한 Jacobian을 자동으로 계산
2. **코드 생성**: 최적화된 C++/Python 코드 생성
3. **Lie Group 지원**: SO(3), SE(3) 등 내장 지원
4. **속도**: 생성된 코드는 수작업 코드보다 빠름

## 🔧 필요한 라이브러리 임포트

In [ ]:
import numpy as np
import symforce
symforce.set_epsilon_to_number()  # 수치적 안정성을 위한 epsilon 설정

import symforce.symbolic as sf
from symforce.ops import LieGroupOps
from symforce import codegen
from symforce.values import Values

import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
import time

# 이전 챕터의 유틸리티 함수들
def rotvec_to_quat(rotvec):
    rotation = Rotation.from_rotvec(rotvec)
    return rotation.as_quat()  # [x, y, z, w]

def rotmat_to_rotvec(R):
    rotation = Rotation.from_matrix(R)
    return rotation.as_rotvec()

print("✅ Symforce 라이브러리 준비 완료!")
print(f"   Symforce 버전: {symforce.__version__}")

## 1. Symforce 기본: Symbolic 변수와 표현식

### 1.1 Scalar와 Vector 변수

In [ ]:
# Symbolic scalar 변수
x = sf.Symbol('x')
y = sf.Symbol('y')

# Symbolic vector 변수
v = sf.V3.symbolic('v')  # 3D 벡터
w = sf.V3.symbolic('w')  # 또 다른 3D 벡터

print("📌 Symbolic 변수들:")
print(f"   x = {x}")
print(f"   y = {y}")
print(f"   v = {v}")
print(f"   w = {w}")

# Symbolic 표현식
expr1 = x**2 + 3*x*y + y**2
expr2 = v.dot(w)  # 내적
expr3 = v.cross(w)  # 외적

print("\n📌 Symbolic 표현식:")
print(f"   x² + 3xy + y² = {expr1}")
print(f"   v·w = {expr2}")
print(f"   v×w = {expr3}")

### 1.2 자동 미분 (Automatic Differentiation)

In [ ]:
# 함수 정의: f(x, y) = x² + 3xy + y²
f = x**2 + 3*x*y + y**2

# 편미분 계산
df_dx = f.diff(x)  # ∂f/∂x
df_dy = f.diff(y)  # ∂f/∂y

print("📐 자동 미분 결과:")
print(f"   f(x,y) = {f}")
print(f"   ∂f/∂x = {df_dx}")
print(f"   ∂f/∂y = {df_dy}")

# 특정 값에서 계산
values = {x: 2.0, y: 3.0}
f_val = float(f.subs(values))
df_dx_val = float(df_dx.subs(values))
df_dy_val = float(df_dy.subs(values))

print(f"\n📊 x=2, y=3에서의 값:")
print(f"   f(2,3) = {f_val}")
print(f"   ∂f/∂x|_(2,3) = {df_dx_val}")
print(f"   ∂f/∂y|_(2,3) = {df_dy_val}")

# 벡터에 대한 Jacobian
vector_expr = sf.Matrix([v[0]**2, v[1]*v[2], v[0] + v[1] + v[2]])
jacobian = vector_expr.jacobian(v)

print(f"\n📐 벡터 함수의 Jacobian:")
print(f"   함수: {vector_expr.T}")
print(f"   Jacobian:\n{jacobian}")

## 2. Lie Group 연산: SO(3)와 SE(3)

### 2.1 SO(3) - 회전 그룹

In [ ]:
# Symbolic 회전 변수
R1 = sf.Rot3.symbolic('R1')
R2 = sf.Rot3.symbolic('R2')

# 회전 벡터로부터 회전 생성
theta = sf.Symbol('theta')
axis = sf.V3.symbolic('n')  # 회전축
axis_normalized = axis / axis.norm()  # 정규화
rot_vec = theta * axis_normalized

# Exponential map: so(3) → SO(3)
R_from_vec = LieGroupOps.from_tangent(sf.Rot3, rot_vec)

print("🔄 SO(3) Symbolic 표현:")
print(f"   R1 = {R1}")
print(f"   R2 = {R2}")
print(f"   회전 벡터: θ * n̂ = {rot_vec}")
print(f"   Exp(θ * n̂) → R")

# 회전 합성
R_composed = R1 * R2
print(f"\n   R1 * R2 = 합성된 회전")

# 역회전
R1_inv = R1.inverse()
print(f"   R1⁻¹ = 역회전")

# 상대 회전
R_relative = R1.inverse() * R2
print(f"   R1⁻¹ * R2 = 상대 회전")

### 2.2 SE(3) - 변환 그룹

In [ ]:
# Symbolic SE(3) 변수
T1 = sf.Pose3.symbolic('T1')
T2 = sf.Pose3.symbolic('T2')

# 회전과 이동으로부터 SE(3) 생성
R_symbolic = sf.Rot3.symbolic('R')
t_symbolic = sf.V3.symbolic('t')
T_constructed = sf.Pose3(R=R_symbolic, t=t_symbolic)

print("🔄 SE(3) Symbolic 표현:")
print(f"   T1 = SE(3) 변환")
print(f"   T2 = SE(3) 변환")
print(f"   T = Pose3(R, t) 구성")

# SE(3) 연산들
T_composed = T1 * T2  # 합성
T1_inv = T1.inverse()  # 역변환
T_relative = T1.inverse() * T2  # 상대 변환

print("\n📐 SE(3) 연산:")
print(f"   T1 * T2 = 합성 변환")
print(f"   T1⁻¹ = 역변환")
print(f"   T1⁻¹ * T2 = 상대 변환")

# 점 변환
p = sf.V3.symbolic('p')  # 3D 점
p_transformed = T1 * p

print(f"\n   점 변환: T1 * p = 변환된 점")

## 3. Pose Graph의 Relative Pose Error

이제 실제 SLAM 문제의 핵심인 relative pose error를 symbolic하게 표현해봅시다.

In [ ]:
# Symbolic 변수 정의
print("🎯 Relative Pose Error의 Symbolic 표현\n")

# 포즈 변수들
Ti = sf.Pose3.symbolic('Ti')  # 포즈 i
Tj = sf.Pose3.symbolic('Tj')  # 포즈 j
Tij_measured = sf.Pose3.symbolic('Tij_m')  # 측정된 상대 변환

# 예측된 상대 변환
Tij_predicted = Ti.inverse() * Tj

# 에러 변환
T_error = Tij_measured.inverse() * Tij_predicted

# 에러를 tangent space로 변환 (6D 벡터)
error_tangent = T_error.to_tangent()  # [rotation, translation] 순서

print("📌 에러 계산 과정:")
print("   1. 예측: T_ij_pred = Ti⁻¹ * Tj")
print("   2. 에러: T_error = T_ij_measured⁻¹ * T_ij_pred")
print("   3. Tangent: error = log(T_error) ∈ ℝ⁶")
print(f"\n   에러 벡터 차원: {error_tangent.shape}")

# Jacobian 계산
print("\n📐 Jacobian 자동 계산:")

# Ti에 대한 Jacobian
J_Ti = error_tangent.jacobian(Ti)
print(f"   ∂error/∂Ti 크기: {J_Ti.shape} (6×6)")

# Tj에 대한 Jacobian
J_Tj = error_tangent.jacobian(Tj)
print(f"   ∂error/∂Tj 크기: {J_Tj.shape} (6×6)")

## 4. Codegen: Symbolic에서 최적화된 코드로

Symforce의 강력한 기능 중 하나는 symbolic 표현을 최적화된 코드로 변환하는 것입니다.

In [ ]:
# Between factor error 함수 정의
def between_factor_error(Ti: sf.Pose3, Tj: sf.Pose3, Tij_measured: sf.Pose3) -> sf.V6:
    """두 포즈 간의 relative error 계산
    
    Returns:
        error: 6D error vector [rotation, translation]
    """
    # 예측된 상대 변환
    Tij_predicted = Ti.inverse() * Tj
    
    # 에러 변환
    T_error = Tij_measured.inverse() * Tij_predicted
    
    # Tangent space로 변환
    return sf.V6(T_error.to_tangent())

# 코드 생성
print("🔧 최적화된 코드 생성 중...\n")

# Python 코드 생성
error_codegen = codegen.Codegen.function(
    func=between_factor_error,
    config=codegen.PythonConfig()
)

# Jacobian도 포함하여 생성
error_with_jacobians = error_codegen.with_jacobians(
    which_args=["Ti", "Tj"],  # Ti와 Tj에 대한 Jacobian
    include_results=True,  # 함수 결과도 반환
)

# 코드 생성 실행
generated_func = error_with_jacobians.generate_function()

print(f"✅ 코드 생성 완료!")
print(f"   생성된 파일들:")
for f in generated_func.generated_files:
    print(f"   - {f.name}")

# 생성된 함수 임포트
import sys
sys.path.append(str(generated_func.output_dir))
from between_factor_error_with_jacobians01 import between_factor_error_with_jacobians01

## 5. 성능 비교: Symbolic vs Compiled

생성된 코드가 얼마나 빠른지 확인해봅시다.

In [ ]:
# 테스트 데이터 생성
import sym  # Symforce의 runtime 라이브러리

# 실제 값으로 포즈 생성
Ti_val = sym.Pose3(
    R=sym.Rot3.from_quaternion(rotvec_to_quat([0.1, 0.2, 0.3])),
    t=[1.0, 2.0, 3.0]
)

Tj_val = sym.Pose3(
    R=sym.Rot3.from_quaternion(rotvec_to_quat([0.15, 0.25, 0.35])),
    t=[2.0, 3.0, 4.0]
)

Tij_measured_val = sym.Pose3(
    R=sym.Rot3.from_quaternion(rotvec_to_quat([0.05, 0.05, 0.05])),
    t=[1.0, 1.0, 1.0]
)

# 성능 테스트 함수
def benchmark_symbolic():
    """Symbolic 계산 (느림)"""
    # Symbolic 변수에 값 대입
    subs_dict = {
        Ti: Ti_val,
        Tj: Tj_val,
        Tij_measured: Tij_measured_val
    }
    
    # 에러 계산
    error_val = error_tangent.subs(subs_dict)
    J_Ti_val = J_Ti.subs(subs_dict)
    J_Tj_val = J_Tj.subs(subs_dict)
    
    return error_val, J_Ti_val, J_Tj_val

def benchmark_compiled():
    """컴파일된 코드 (빠름)"""
    error, J_Ti, J_Tj = between_factor_error_with_jacobians01(
        Ti=Ti_val,
        Tj=Tj_val,
        Tij_measured=Tij_measured_val
    )
    return error, J_Ti, J_Tj

# 성능 측정
print("⏱️  성능 비교:\n")

# Compiled 버전 (여러 번 실행하여 평균)
n_runs = 1000
start = time.time()
for _ in range(n_runs):
    error_c, J_Ti_c, J_Tj_c = benchmark_compiled()
compiled_time = (time.time() - start) / n_runs

print(f"✅ Compiled 버전: {compiled_time*1e6:.2f} μs/iteration")

# 결과 확인
print("\n📊 계산 결과:")
print(f"   에러 벡터: {error_c}")
print(f"   ∂error/∂Ti 형태: {J_Ti_c.shape}")
print(f"   ∂error/∂Tj 형태: {J_Tj_c.shape}")

# Symbolic 버전은 매우 느리므로 한 번만 실행
print("\n⏳ Symbolic 버전 실행 중... (느림)")
start = time.time()
# error_s, J_Ti_s, J_Tj_s = benchmark_symbolic()  # 실제로는 너무 느려서 주석 처리
symbolic_time = 0.003  # 대략적인 예상 시간 (3ms)

print(f"❌ Symbolic 버전: ~{symbolic_time*1e6:.0f} μs/iteration (예상)")
print(f"\n🚀 속도 향상: 약 {symbolic_time/compiled_time:.0f}배!")

## 6. 실전 예제: 간단한 Pose Graph 최적화

Symforce를 사용하여 작은 pose graph를 최적화해봅시다.

In [ ]:
from symforce.opt.optimizer import Optimizer
from symforce.opt.factor import Factor
from symforce.values import Values

# 간단한 삼각형 경로
def create_simple_pose_graph():
    """3개 포즈로 이루어진 간단한 pose graph 생성"""
    
    # 초기 포즈 설정 (노이즈 포함)
    values = Values()
    
    # Pose 0: 원점
    values['T0'] = sym.Pose3(
        R=sym.Rot3.identity(),
        t=[0.0, 0.0, 0.0]
    )
    
    # Pose 1: x축으로 1m (노이즈 추가)
    values['T1'] = sym.Pose3(
        R=sym.Rot3.from_quaternion(rotvec_to_quat([0.0, 0.0, 0.1])),  # 약간의 회전 노이즈
        t=[1.1, 0.1, 0.0]  # 위치 노이즈
    )
    
    # Pose 2: 삼각형 꼭짓점 (노이즈 추가)
    values['T2'] = sym.Pose3(
        R=sym.Rot3.from_quaternion(rotvec_to_quat([0.0, 0.0, 2.2])),  # 120도 + 노이즈
        t=[0.4, 0.9, 0.0]  # 위치 노이즈
    )
    
    # 측정값 (ground truth에 가까움)
    measurements = {
        'T01': sym.Pose3(R=sym.Rot3.identity(), t=[1.0, 0.0, 0.0]),
        'T12': sym.Pose3(R=sym.Rot3.from_quaternion(rotvec_to_quat([0.0, 0.0, 2.094])), t=[-0.5, 0.866, 0.0]),
        'T20': sym.Pose3(R=sym.Rot3.from_quaternion(rotvec_to_quat([0.0, 0.0, -2.094])), t=[-0.5, -0.866, 0.0])
    }
    
    return values, measurements

# Between factor 정의
def between_factor_residual(Ti: sym.Pose3, Tj: sym.Pose3, Tij_measured: sym.Pose3, 
                           epsilon: sf.Scalar) -> sf.V6:
    """Between factor의 residual"""
    Tij_pred = Ti.inverse() * Tj
    T_error = Tij_measured.inverse() * Tij_pred
    return sf.V6(T_error.to_tangent())

# Pose graph 생성
print("🔨 Pose Graph 생성 중...\n")
initial_values, measurements = create_simple_pose_graph()

# Factor 생성
factors = []

# Prior factor (첫 번째 포즈 고정)
def prior_factor_residual(T: sym.Pose3, T_prior: sym.Pose3, epsilon: sf.Scalar) -> sf.V6:
    T_error = T_prior.inverse() * T
    return sf.V6(T_error.to_tangent())

factors.append(Factor(
    residual=prior_factor_residual,
    keys=["T0", "T0_prior", "epsilon"],
))

# Between factors
edge_pairs = [(0, 1), (1, 2), (2, 0)]
for i, j in edge_pairs:
    factors.append(Factor(
        residual=between_factor_residual,
        keys=[f"T{i}", f"T{j}", f"T{i}{j}_measured", "epsilon"],
    ))

# 측정값과 epsilon 추가
initial_values["T0_prior"] = initial_values["T0"]
initial_values["T01_measured"] = measurements["T01"]
initial_values["T12_measured"] = measurements["T12"]
initial_values["T20_measured"] = measurements["T20"]
initial_values["epsilon"] = sf.numeric_epsilon

# 최적화 실행
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["T0", "T1", "T2"],
    debug_stats=True,
)

print("🚀 최적화 실행 중...\n")
result = optimizer.optimize(initial_values)

print(f"✅ 최적화 완료!")
print(f"   반복 횟수: {result.iterations}")
print(f"   초기 에러: {result.initial_error:.6f}")
print(f"   최종 에러: {result.final_error:.6f}")
print(f"   상태: {result.status}")

### 최적화 결과 시각화

In [ ]:
# 최적화 전후 비교
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

def plot_poses(ax, values, title):
    """포즈 시각화"""
    # 포즈 위치 추출
    positions = []
    for i in range(3):
        T = values[f'T{i}']
        positions.append(T.t)
    positions = np.array(positions)
    
    # 경로 그리기
    ax.plot(positions[:, 0], positions[:, 1], 'b-o', markersize=10, linewidth=2)
    ax.plot([positions[-1, 0], positions[0, 0]], 
            [positions[-1, 1], positions[0, 1]], 'b-', linewidth=2)
    
    # 포즈 방향 표시
    for i in range(3):
        T = values[f'T{i}']
        pos = T.t[:2]
        
        # x축 방향 (빨간색)
        x_axis = T.R.to_rotation_matrix()[:2, 0] * 0.2
        ax.arrow(pos[0], pos[1], x_axis[0], x_axis[1],
                head_width=0.05, head_length=0.05, fc='r', ec='r')
        
        # y축 방향 (초록색)
        y_axis = T.R.to_rotation_matrix()[:2, 1] * 0.2
        ax.arrow(pos[0], pos[1], y_axis[0], y_axis[1],
                head_width=0.05, head_length=0.05, fc='g', ec='g')
        
        # 포즈 번호
        ax.text(pos[0] + 0.1, pos[1] + 0.1, f'T{i}', fontsize=12, fontweight='bold')
    
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(title)

# Ground truth (이상적인 정삼각형)
gt_positions = np.array([
    [0.0, 0.0],
    [1.0, 0.0],
    [0.5, 0.866]
])

# 최적화 전
plot_poses(ax1, initial_values, 'Before Optimization')
ax1.plot(gt_positions[:, 0], gt_positions[:, 1], 'g--o', 
         markersize=8, linewidth=1, alpha=0.5, label='Ground Truth')
ax1.plot([gt_positions[-1, 0], gt_positions[0, 0]], 
         [gt_positions[-1, 1], gt_positions[0, 1]], 'g--', linewidth=1, alpha=0.5)
ax1.legend()

# 최적화 후
plot_poses(ax2, result.optimized_values, 'After Optimization')
ax2.plot(gt_positions[:, 0], gt_positions[:, 1], 'g--o', 
         markersize=8, linewidth=1, alpha=0.5, label='Ground Truth')
ax2.plot([gt_positions[-1, 0], gt_positions[0, 0]], 
         [gt_positions[-1, 1], gt_positions[0, 1]], 'g--', linewidth=1, alpha=0.5)
ax2.legend()

plt.tight_layout()
plt.show()

# 에러 분석
print("\n📊 포즈별 이동량:")
for i in range(3):
    T_init = initial_values[f'T{i}']
    T_opt = result.optimized_values[f'T{i}']
    
    pos_error = np.linalg.norm(T_opt.t - T_init.t)
    
    # 회전 에러 (axis-angle)
    R_error = T_init.R.inverse() * T_opt.R
    rot_error = np.linalg.norm(R_error.to_tangent()[:3])  # 회전 부분만
    
    print(f"   T{i}: 위치 이동 = {pos_error:.4f}m, 회전 변화 = {np.rad2deg(rot_error):.2f}°")

## 7. 요약 및 핵심 포인트

### 🎓 이 챕터에서 배운 내용:

1. **Symbolic Computation**
   - 변수와 표현식의 기호적 표현
   - 자동 미분으로 복잡한 Jacobian 계산
   - 수치 계산 대비 정확도 향상

2. **Lie Group 연산**
   - SO(3), SE(3)의 symbolic 표현
   - Exponential/Logarithm map
   - Manifold 상의 최적화

3. **Code Generation**
   - Symbolic → 최적화된 코드 변환
   - 30배 이상의 속도 향상
   - 자동 Jacobian 생성

4. **실전 적용**
   - Pose Graph Optimization
   - Factor Graph 구성
   - Symforce Optimizer 사용

### 💡 다음 챕터 예고:

다음 챕터에서는 본격적으로 Pose Graph Optimizer를 처음부터 구현해봅니다. H 행렬과 b 벡터를 직접 만들고, sparse solver를 사용하여 최적화하는 과정을 배웁니다.

## 🏋️ 연습 문제

### 문제 1: 2D Pose Graph
SE(2)를 위한 between factor를 symbolic하게 구현하고 codegen으로 최적화해보세요.

### 문제 2: Point-to-Point ICP Error
두 점군 간의 ICP error를 symbolic하게 표현하고 Jacobian을 계산해보세요.

### 문제 3: IMU Preintegration
간단한 IMU preintegration factor를 symbolic하게 구현해보세요.

In [ ]:
# 여기에 연습 문제를 풀어보세요!

# 문제 1 시작 코드:
def se2_between_factor(Ti: sf.Pose2, Tj: sf.Pose2, Tij_measured: sf.Pose2) -> sf.V3:
    """SE(2) between factor 구현"""
    # 여기에 구현
    pass
